In [ ]:
import requests
import pprint as pp
import pandas as pd
import numpy as np
import gmaps
import pycountry

# Import API key
from api_keys import g_key

#Output Folder Path
path = "output_data/"

# Access maps with unique API key
gmaps.configure(api_key=g_key)

#Read_csv_Path
uscity_path = f"{path}player_stats_2022_uscity_clean.csv"
intcity_path = f"{path}player_stats_2022_intcity_clean.csv"
country_path = f"{path}player_stats_2022_country_clean.csv"
college_path = f"{path}player_stats_2022_college_clean.csv"

In [ ]:
#Read CSV
uscity_df = pd.read_csv(uscity_path).drop_duplicates()
intcity_df = pd.read_csv(intcity_path).drop_duplicates()
country_df = pd.read_csv(country_path).drop_duplicates()
college_df = pd.read_csv(college_path).drop_duplicates()

print(list(uscity_df))
print(list(intcity_df))
print(list(country_df))
print(list(college_df))

uscity_df

## Lat/Longs

In [ ]:
# US cities

url = f"https://maps.googleapis.com/maps/api/geocode/json?"

params = {
    'key': g_key
}

for i in uscity_df.index:
    city = uscity_df.loc[i,'Player US Full City']
    params['address'] = city
    
    response = requests.get(url, params=params).json()
        
    try:
        result = response['results'][0]
        print(f'Processing {city}')
        uscity_df.loc[i,'Lat'] = result['geometry']['location']['lat']
        uscity_df.loc[i,'Lng'] = result['geometry']['location']['lng']
    except Exception as e:
        print(e)

uscity_df.to_csv('output_data/uscity_coord.csv')
uscity_df


In [ ]:
# International cities

url = f"https://maps.googleapis.com/maps/api/geocode/json?"

params = {
    'key': g_key
}

for i in intcity_df.index:
    city = intcity_df.loc[i,'Player Int Full City']
    params['address'] = city
    
    response = requests.get(url, params=params).json()
        
    try:
        result = response['results'][0]
        print(f'Processing {city}')
        intcity_df.loc[i,'Lat'] = result['geometry']['location']['lat']
        intcity_df.loc[i,'Lng'] = result['geometry']['location']['lng']
    except Exception as e:
        print(e)

intcity_df.to_csv('output_data/intcity_coord.csv')
intcity_df

## Maps by US Cities

In [ ]:
uscity_players = uscity_df.groupby('Player City')['Player'].count()
uscity_players

In [ ]:
#Create a list containing coordinates
coordinates = [(uscity_df[i, 'Lat'], uscity_df[i,'Lng']) for i in uscity_df.index]

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#Creating the figure holder
fig = gmaps.figure(map_type='HYBRID', layout=figure_layout)

#Assign the heatmap layer to a variable
heatmap = gmaps.heatmap_layer(coordinates, dissipating=False, point_radius=5)

#Add the layer to the map
fig.add_layer(heatmap)

#Display figure
fig


In [ ]:
# Using the template add marks to the heatmap
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Qty Players</dt><dd>{Players}</dd>
</dl>
"""

#Data retrieval for Latitude and Longitude
players=[]
top_city=[]

top_city_index = players_uscity['Players'].idxmax()
top_city = players_uscity.loc[top_city_index,'US City']
lat = players_uscity_gloc['Lat'].loc[players_uscity_gloc['US City'] == top_city]
lng = players_uscity_gloc['Lng'].loc[players_uscity_gloc['US City'] == top_city]
players.append(players_uscity.loc[top_city_index,'Players'])

#Dataframe creation for map display
data={"City":top_city, "Players":players ,"Lat":lat, "Lng":lng}
df= pd.DataFrame(data)

# Store the DataFrame Row
marker_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["Lat", "Lng"]]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
symbol_layer = gmaps.symbol_layer(locations, info_box_content=marker_info)

# Add the layer to the map
fig = gmaps.figure()
fig.add_layer(markers)
fig.add_layer(heatmap)
fig.add_layer(symbol_layer)


# Display figure
fig


## Maps by International Cities

In [ ]:
#Create a list containing coordinates
coordinates = list(zip(players_intcity_gloc["Lat"],players_intcity_gloc["Lng"]))

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#Creating the figure holder
fig = gmaps.figure(layout=figure_layout)

# Assign the heatmap layer to a variable
heatmap = gmaps.heatmap_layer(coordinates, weights=players_intcity["Players"])

# Add the layer to the map
fig.add_layer(heatmap)

#Display figure
fig


In [ ]:
# Using the template add marks to the heatmap
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Qty Players</dt><dd>{Players}</dd>
</dl>
"""

#Data retrieval for Latitude and Longitude
players=[]
top_city=[]

top_city_index = players_intcity['Players'].idxmax()
top_city = players_intcity.loc[top_city_index,'Int City']
lat = players_intcity_gloc['Lat'].loc[players_intcity_gloc['Int City'] == top_city]
lng = players_intcity_gloc['Lng'].loc[players_intcity_gloc['Int City'] == top_city]
players.append(players_intcity.loc[top_city_index,'Players'])

data={"City":top_city, "Players":players ,"Lat":lat, "Lng":lng}
df= pd.DataFrame(data)

# Store the DataFrame Row
marker_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["Lat", "Lng"]]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
symbol_layer = gmaps.symbol_layer(locations, info_box_content=marker_info)

# Add the layer to the map
fig = gmaps.figure()
fig.add_layer(markers)
fig.add_layer(heatmap)
fig.add_layer(symbol_layer)


# Display figure
fig


## Maps by Countries

In [ ]:
#Create a list containing coordinates
coordinates = list(zip(players_countries_gloc["Lat"],players_countries_gloc["Lng"]))

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#Creating the figure holder
fig = gmaps.figure(layout=figure_layout)

# Assign the heatmap layer to a variable
heatmap = gmaps.heatmap_layer(coordinates, weights=players_countries["Players"])

# Add the layer to the map
fig.add_layer(heatmap)

#Display figure
fig

In [ ]:
# Using the template add marks to the heatmap
info_box_template = """
<dl>
<dt>Country</dt><dd>{Country}</dd>
<dt>Qty Players</dt><dd>{Players}</dd>
</dl>
"""

#Data retrieval for Latitude and Longitude
players=[]
top_country=[]

top_country_index = players_countries['Players'].idxmax()
top_country_data = players_countries.loc[top_country_index,'Country']
top_country_conv = pycountry.countries.lookup(top_country_data)
top_country = top_country_conv.name

lat = players_countries_gloc['Lat'].loc[players_countries_gloc['Country'] == top_country]
lng = players_countries_gloc['Lng'].loc[players_countries_gloc['Country'] == top_country]
players.append(players_countries.loc[top_country_index,'Players'])

data={"Country":top_country, "Players":players ,"Lat":lat, "Lng":lng}
df= pd.DataFrame(data)

# Store the DataFrame Row
marker_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["Lat", "Lng"]]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
symbol_layer = gmaps.symbol_layer(locations, info_box_content=marker_info)

# Add the layer to the map
fig = gmaps.figure()
fig.add_layer(markers)
fig.add_layer(heatmap)
fig.add_layer(symbol_layer)


# Display figure
fig
